In the first notebook, a table was created that described the distance between points at every 25 meters on each modeling network segment. This tabe will be the guide to conflating the two networks. The conflation will start with divided highways and then conflate lower class segments. This process uses only the pandas module, and a variety of sorting methods to determine 'good' matches.

In [4]:
import pandas as pd
import os
import arcpy as ap


path_to_data = os.getcwd() + '\\Data\\'

In [5]:
modeling_attributes_df = pd.read_csv(path_to_data + 'modeling_attributes.csv')
modeling_type = modeling_attributes_df.filter(['modeling_id', 'modeling_type'])

modeling_not_highways = modeling_type.query('modeling_type != 1')
modeling_not_highways_nor_ramps  = modeling_not_highways.query('modeling_type != 6')

list_of_modeling_not_highways_nor_ramps = list(modeling_not_highways_nor_ramps['modeling_id'].unique())

del modeling_type, modeling_attributes_df, modeling_not_highways

modeling_not_highways_nor_ramps.head()

,modeling_id,modeling_type
1,1,4
3,3,3
4,4,4
5,5,4
6,6,3


In [6]:
osm_attributes_df = pd.read_csv(path_to_data + 'osm_attributes.csv')
osm_type = osm_attributes_df.filter(['osm_id', 'osm_type'])

osm_not_highways = osm_attributes_df[osm_attributes_df['osm_type'] != 'motorway']
osm_not_highways_nor_ramps = osm_not_highways[osm_not_highways['osm_type'] != 'motorway_link']
list_of_osm_not_highways_nor_ramps = list(osm_not_highways_nor_ramps['osm_id'])

del osm_attributes_df, osm_type, osm_not_highways

osm_not_highways_nor_ramps.head()

,osm_id,osm_name,osm_type,osm_bearing
1,1,Park Meadows Boulevard,tertiary,190.15
2,2,Midway Boulevard,tertiary,98.66
6,6,North Cleveland Avenue,primary,184.66
7,7,Cleveland Avenue,residential,270.85
8,8,Cleveland Avenue,secondary,270.05


In [7]:
matches = pd.read_csv(path_to_data + 'matches.csv')
matches = matches.filter(['modeling_id', 'osm_id', 'NEAR_RANK', 'NEAR_ANGLE','distance', 'FROM_X', 'FROM_Y', 'NEAR_X', 'NEAR_Y'])

matches.head()

,modeling_id,osm_id,NEAR_RANK,NEAR_ANGLE,distance,FROM_X,FROM_Y,NEAR_X,NEAR_Y
0,0,8163,1,70.350252,4.858731,-104.038302,39.610682,-104.038249,39.610697
1,0,8120,2,71.112462,41.678034,-104.038302,39.610682,-104.037843,39.610804
2,0,8163,1,73.996497,7.919202,-104.038114,39.610135,-104.038025,39.610155
3,0,8120,2,71.112581,45.998868,-104.038114,39.610135,-104.037607,39.610269
4,0,8163,1,75.831732,8.581458,-104.037930,39.609586,-104.037833,39.609605


In [8]:
not_highway_modeling_matches = matches[matches['modeling_id'].isin(list_of_modeling_not_highways_nor_ramps)]

modeling_not_highways_matches_osm_not_highways_df = not_highway_modeling_matches[not_highway_modeling_matches['osm_id'].isin(
                                               list_of_osm_not_highways_nor_ramps)]

#modeling_not_highways_matches_osm_not_highways_df = modeling_not_highways_matches_osm_not_highways_df.query('NEAR_RANK == 1')

modeling_not_highways_matches_osm_not_highways_df.head()

,modeling_id,osm_id,NEAR_RANK,NEAR_ANGLE,distance,FROM_X,FROM_Y,NEAR_X,NEAR_Y
262,1,4266,2,-117.450642,16.354786,-104.010537,39.560530,-104.010706,39.560462
265,1,4266,2,-117.556075,15.418521,-104.010869,39.561005,-104.011028,39.560940
267,1,4266,1,-117.556285,14.538486,-104.011200,39.561479,-104.011350,39.561418
270,1,4266,1,-117.402230,13.596503,-104.011531,39.561953,-104.011672,39.561897
273,1,4266,1,-117.402440,12.556092,-104.011863,39.562428,-104.011992,39.562376


In [9]:
not_highway_matches = modeling_not_highways_matches_osm_not_highways_df

modeling_attributes = pd.read_csv(path_to_data + 'modeling_attributes.csv')
osm_attributes = pd.read_csv(path_to_data + 'osm_attributes.csv')

not_highway_matches_with_attributes = pd.merge(not_highway_matches, 
                                                    modeling_attributes, 
                                                    on = 'modeling_id')

not_highway_matches_with_attributes = pd.merge(not_highway_matches_with_attributes, 
                                                        osm_attributes, 
                                                        on = 'osm_id')

not_highway_matches_with_attributes.head()

,modeling_id,osm_id,NEAR_RANK,NEAR_ANGLE,distance,FROM_X,FROM_Y,NEAR_X,NEAR_Y,modeling_name,modeling_type,modeling_bearing,osm_name,osm_type,osm_bearing
0,1,4266,2,-117.450642,16.354786,-104.010537,39.560530,-104.010706,39.560462,Sh-40 East,4,325.88,State Highway 40,secondary,146.09
1,1,4266,2,-117.556075,15.418521,-104.010869,39.561005,-104.011028,39.560940,Sh-40 East,4,325.88,State Highway 40,secondary,146.09
2,1,4266,1,-117.556285,14.538486,-104.011200,39.561479,-104.011350,39.561418,Sh-40 East,4,325.88,State Highway 40,secondary,146.09
3,1,4266,1,-117.402230,13.596503,-104.011531,39.561953,-104.011672,39.561897,Sh-40 East,4,325.88,State Highway 40,secondary,146.09
4,1,4266,1,-117.402440,12.556092,-104.011863,39.562428,-104.011992,39.562376,Sh-40 East,4,325.88,State Highway 40,secondary,146.09


In [10]:
def getDifference(b1, b2):
    r = (b2 - b1) % 360.0
    # Python modulus has same sign as divisor, which is positive here,
    # so no need to consider negative case
    if r >= 180.0:
        r -= 360.0
    return r

In [11]:
matches_with_attributes = not_highway_matches_with_attributes

#del not_highway_matches_with_attributes

matches_with_attributes['bearing_diff'] = matches_with_attributes.apply(
                                                          lambda x: 
                                                          abs(getDifference(x['osm_bearing'], 
                                                                            x['modeling_bearing'])), 
                                                          axis=1
                                                          )

matches_with_attributes = matches_with_attributes[
                                                (matches_with_attributes['bearing_diff'] < 45) | 
                                                (matches_with_attributes['bearing_diff'] > 135)
                                                ]

matches_with_attributes.head()

,modeling_id,osm_id,NEAR_RANK,NEAR_ANGLE,distance,FROM_X,FROM_Y,NEAR_X,NEAR_Y,modeling_name,modeling_type,modeling_bearing,osm_name,osm_type,osm_bearing,bearing_diff
0,1,4266,2,-117.450642,16.354786,-104.010537,39.560530,-104.010706,39.560462,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79
1,1,4266,2,-117.556075,15.418521,-104.010869,39.561005,-104.011028,39.560940,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79
2,1,4266,1,-117.556285,14.538486,-104.011200,39.561479,-104.011350,39.561418,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79
3,1,4266,1,-117.402230,13.596503,-104.011531,39.561953,-104.011672,39.561897,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79
4,1,4266,1,-117.402440,12.556092,-104.011863,39.562428,-104.011992,39.562376,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79


In [12]:
pd.options.mode.chained_assignment = None  # default='warn'

matches_with_attributes['near_diff'] = matches_with_attributes.apply(
                                                       lambda x:
                                                       abs(getDifference(x['NEAR_ANGLE'],
                                                                         x['modeling_bearing'])),
                                                       axis=1
                                                       )

In [13]:
matches_with_attributes.head()

,modeling_id,osm_id,NEAR_RANK,NEAR_ANGLE,distance,FROM_X,FROM_Y,NEAR_X,NEAR_Y,modeling_name,modeling_type,modeling_bearing,osm_name,osm_type,osm_bearing,bearing_diff,near_diff
0,1,4266,2,-117.450642,16.354786,-104.010537,39.560530,-104.010706,39.560462,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.330642
1,1,4266,2,-117.556075,15.418521,-104.010869,39.561005,-104.011028,39.560940,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.436075
2,1,4266,1,-117.556285,14.538486,-104.011200,39.561479,-104.011350,39.561418,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.436285
3,1,4266,1,-117.402230,13.596503,-104.011531,39.561953,-104.011672,39.561897,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.282230
4,1,4266,1,-117.402440,12.556092,-104.011863,39.562428,-104.011992,39.562376,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.282440


In [14]:
matches_with_attributes = matches_with_attributes[
                                                (matches_with_attributes['near_diff'] > 75) &
                                                (matches_with_attributes['near_diff'] < 105)
                                                ]
matches_with_attributes.head()

,modeling_id,osm_id,NEAR_RANK,NEAR_ANGLE,distance,FROM_X,FROM_Y,NEAR_X,NEAR_Y,modeling_name,modeling_type,modeling_bearing,osm_name,osm_type,osm_bearing,bearing_diff,near_diff
0,1,4266,2,-117.450642,16.354786,-104.010537,39.560530,-104.010706,39.560462,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.330642
1,1,4266,2,-117.556075,15.418521,-104.010869,39.561005,-104.011028,39.560940,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.436075
2,1,4266,1,-117.556285,14.538486,-104.011200,39.561479,-104.011350,39.561418,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.436285
3,1,4266,1,-117.402230,13.596503,-104.011531,39.561953,-104.011672,39.561897,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.282230
4,1,4266,1,-117.402440,12.556092,-104.011863,39.562428,-104.011992,39.562376,Sh-40 East,4,325.88,State Highway 40,secondary,146.09,179.79,83.282440


In [15]:
matches_with_attributes['FID'] = matches_with_attributes.index

matches_with_attributes.to_csv(path_to_data + 'not_highway_match_lines.csv')

ap.Delete_management(path_to_data + 'not_highway_match_lines.shp')

ap.XYToLine_management(path_to_data + 'not_highway_match_lines.csv', 
                         path_to_data + 'not_highway_match_lines.shp',
                         'FROM_X',
                         'FROM_Y',
                         'NEAR_X',
                         'NEAR_Y',
                         'GEODESIC',
                         'FID')

ap.AddGeometryAttributes_management(path_to_data + 'not_highway_match_lines.shp', 'LENGTH_GEODESIC')

<Result 'C:\\Users\\mcintyrei\\Desktop\\Definitely\\OSM_Conflation\\Data\\not_highway_match_lines.shp'>